# ロジスティック回帰の正則化オプション指定例

scikit-learn のロジスティック回帰モジュール  に正規化オプションを指定し、実行させる例を下記に掲載いたします。

使い方としては、


- class_weight（インスタンス生成時引数）

  クラスごとに重み付けを与える
  （クラス＝質問文から選択された回答文と等価）


- sample_weight（学習実行時引数）

  訓練データごとに重み付けを与える
  （訓練データ＝質問文１件と等価）


となるかと存じます。


こちらの例では、効果がわかりやすくなるように、あえて極端な例<b>（質問に対し、一定の条件で回答が強要または回避されるような例）</b>を設定して実行させています。

なにとぞ、ご容赦ください。

## (0) テスト準備

長いので適宜読み飛ばしていただければ幸いです

In [1]:
# 簡単にするために、３分類のテストデータを使います。
corpus = []

# 質問分類１
corpus.append('Do you like an apple ?')
corpus.append('Do you like an orange ?')
corpus.append('Do you like a lemon ?')


# 質問分類２
corpus.append('Does he have a note ?')
corpus.append('Does she have a paper ?')
corpus.append('Does he have a watch ?')


# 質問分類３
corpus.append('Did she get a computer ?')
corpus.append('Did you get a camera ?')
corpus.append('Did he get a car ?')


corpus

['Do you like an apple ?',
 'Do you like an orange ?',
 'Do you like a lemon ?',
 'Does he have a note ?',
 'Does she have a paper ?',
 'Does he have a watch ?',
 'Did she get a computer ?',
 'Did you get a camera ?',
 'Did he get a car ?']

In [2]:
tf_feature_names = []
tf_feature_names.append('like')
tf_feature_names.append('have')
tf_feature_names.append('get')
tf_feature_names.append('a')
tf_feature_names.append('an')

tf_feature_names # これは特徴語のリストです。

['like', 'have', 'get', 'a', 'an']

In [3]:
from numpy import array
from scipy.sparse.csr import csr_matrix
tf_vector_array = array([
    [1, 0, 0, 0, 1],
    [1, 0, 0, 0, 1],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 1, 1, 0]
    ])
tf_vector = csr_matrix(tf_vector_array) # これは特徴語の出現回数リストです。
tf_vector.toarray()

array([[1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 0]], dtype=int64)

In [4]:
class_vector_array = array([
    1,
    1,
    1,
    2,
    2,
    2,
    3,
    3,
    3
    ])
class_vector_array # これは質問文ごとのクラスリスト（分類ラベルリスト）です。

array([1, 1, 1, 2, 2, 2, 3, 3, 3])

## (1) class_weight を指定した時の動き

#### class_weight の指定

クラスごとに重み付け定義を指定します。

In [5]:
class_weight = {# <---ご注意：class_weight は dictionary です
    1: 1.0, 
    2: 2.0, # <---他のクラスに対して、２倍の重み付けがされています
    3: 1.0
    }
class_weight # これはクラスごとの重み付け定義リストです。

{1: 1.0, 2: 2.0, 3: 1.0}

#### インスタンスを生成→学習実行

class_weight を指定し、学習を実行します。

In [6]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(
                        penalty ='l2', # <---L2正則化が効くようにします
                        C       = 0.1, # <---正則化の効きを大きくします
                        class_weight=class_weight, # <---重み付け定義を指定
                        max_iter=100,
                        solver='newton-cg',
                        )
logisticRegression.fit(tf_vector, class_vector_array)

LogisticRegression(C=0.1, class_weight={1: 1.0, 2: 2.0, 3: 1.0}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='newton-cg', tol=0.0001, verbose=0, warm_start=False)

#### 質問文を作成

訓練データに近い内容にしておきます

In [7]:
tf_vector_array_test = array([
    [1.1, 0, 0, 0, 1], # これはクラス＝１想定
    [0, 1.2, 0, 0, 1], # これはクラス＝２想定
    [0, 0, 1.3, 0, 0]  # これはクラス＝３想定
    ])
tf_vector_test = csr_matrix(tf_vector_array_test) # これは特徴語の出現回数リストです。
tf_vector_test.toarray()

array([[ 1.1,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  1.2,  0. ,  0. ,  1. ],
       [ 0. ,  0. ,  1.3,  0. ,  0. ]])

#### 予測の実行

  全ての質問文に対して、重み付けをした２番目のクラスに<b>回答が強要されてしまう</b>ことが確認できます。

In [8]:
Zp = logisticRegression.predict_proba(tf_vector_test)
Zp

array([[ 0.35075129,  0.41900842,  0.23024029],
       [ 0.26393346,  0.52531244,  0.2107541 ],
       [ 0.25775191,  0.43278232,  0.30946577]])

## (2) sample_weight を指定した時の動き

#### sample_weight の指定

訓練データごとに重み付け定義を指定します。

In [9]:
sample_weight = array([ # <---ご注意：sample_weight は array です
    1.0,
    1.0,
    0.1, # [1, 0, 0, 1, 0] という特徴を持った訓練データ（クラス＝１）の重み付けを下げます
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0
    ])
sample_weight # これは訓練データごとの重み付け定義リストです。

array([ 1. ,  1. ,  0.1,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ])

#### インスタンスを生成→学習

sample_weight を指定し、学習を実行します。

In [10]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(
                        penalty ='l2', # <---L2正則化が効くようにします
                        C       = 0.1, # <---正則化の効きを大きくします
                        class_weight=None, # <---クラスごとの重み付け定義は指定しない
                        max_iter=100,
                        solver='newton-cg',
                        )
logisticRegression.fit(tf_vector, class_vector_array, sample_weight=sample_weight)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

#### 質問文を作成

訓練データに近い内容にしておきます

In [11]:
tf_vector_array_test = array([
    [1.1, 0, 0, 1, 0],  # これはクラス＝１想定（重み付けを下げた特徴を持っている）
    [0, 1.2, 0, 0, 1],  # これはクラス＝２想定
    [0, 0, 1.3, 0, 0]   # これはクラス＝３想定
    ])
tf_vector_test = csr_matrix(tf_vector_array_test) # これは特徴語の出現回数リストです。
tf_vector_test.toarray()

array([[ 1.1,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  1.2,  0. ,  0. ,  1. ],
       [ 0. ,  0. ,  1.3,  0. ,  0. ]])

#### 予測の実行

本来、１番目の質問文はクラス１と回答されるところが、sample_weight 指定により<b>回答が回避されてしまう</b>ことが確認できます。

他の質問文は、概ね想定通りに分類されます。

In [12]:
Zp = logisticRegression.predict_proba(tf_vector_test)
Zp

array([[ 0.27975145,  0.36012427,  0.36012427],
       [ 0.28505076,  0.39608815,  0.31886109],
       [ 0.25604088,  0.32955323,  0.41440589]])